In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

In [2]:
tour_data = pd.read_csv('/Users/jeenskki/Documents/bigdata2020/data/tour_data.csv', encoding='UTF-8')

In [3]:
tour_data["title"] = tour_data["title"].str.replace("[()\[\]㈜]", "")
tour_data["title2"] = tour_data["title"].str.replace("서울", " ")
tour_data["title2"] = tour_data["title2"].str.replace(" 대학교", "서울대학교")
tour_data["title2"] = tour_data["title2"].str.replace(" ", ",")

tour_data["title2"].head(10)

0               코스테이
1      피에프비파트너스,마린칸토
2             가나아트센터
3        가나안경원,명동2호점
4      가든파이브,Garden5
5             가락관광호텔
6    가락시장가락농수산물,도매시장
7               가람화랑
8          가로수길,홈스테이
9             가로수스테이
Name: title2, dtype: object

In [11]:
import time

def crawling(tour_data):
    val = []
    # driver 경로는 각자 PC 경로로 할 것
    driver = webdriver.Chrome("/Users/jeenskki/Desktop/chromedriver_mac64/chromedriver")
    driver.get("https://datalab.naver.com/keyword/trendSearch.naver")
    tries = 0
    for i in range(len(tour_data)):
        while True:
            try:
                driver.get("https://datalab.naver.com/keyword/trendSearch.naver") 
                driver.find_element_by_id("item_keyword1").send_keys(tour_data["title"].iloc[i])
                driver.find_element_by_id("item_sub_keyword1_1").send_keys(tour_data["title2"].iloc[i])

                driver.find_element_by_id("timeDimensionTitle").click()
                driver.find_element_by_class_name("_item_month").click()
                driver.find_element_by_css_selector("._trend_search_detail_query").click()    

                res = []

                time.sleep(1)

                for j in range(1, 13):
                    action = ActionChains(driver)
                    c = driver.find_element_by_css_selector("circle.bb-circle:nth-of-type({})".format(j))
                    action.move_to_element(c).perform()

                    cnt = driver.find_element_by_css_selector(".tooltip > .value").get_attribute("innerHTML")
                    res.append(cnt)        
                val.append(res)
                print("Sequence Number : {}\r".format(i), end="")
                
            except NoSuchElementException:
                if tries < 10:
                    print("The Exception has been occured. Activating time sleep...")
                    time.sleep(600)
                    print("10 minutes passed. Let's retry !")
                    tries += 1
                    continue
                else:
                    print("The system has encountered maximum retries. Aborting loop...")
                    break
            break
            
    driver.quit()
    
    col = []
    for i in range(1, 13):
        v = "month_{}".format(i)
        col.append(v)
        
    df = pd.DataFrame(val, columns = col)
    df["tour_id"] = tour_data["tour_id"]
    df = df[-1:] + df[:-1]
    
    return df

In [10]:
result = crawling(tour_data)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=86.0.4240.183)
